# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Use csv from WeatherPy
weather_data = pd.read_csv("../WeatherPy/output/weather_data.csv")

# Display first five rows of csv
weather_data.head()

,city,longitude,latitude,maxtemp_f,humidity_percent,cloudiness_percent,windspeed_mph,country,date
0,San Isidro,-77.0500,-12.1167,68.50,50.0,0.0,1.01,PE,2022-02-05 22:20:17-05:00
1,Ushuaia,-68.3000,-54.8000,49.66,57.0,20.0,18.41,AR,2022-02-05 22:16:47-05:00
2,Tasiilaq,-37.6368,65.6145,14.16,55.0,95.0,6.93,GL,2022-02-05 22:10:33-05:00
3,Key Largo,-80.4473,25.0865,72.01,80.0,48.0,8.88,US,2022-02-05 22:19:45-05:00
4,Norman Wells,-126.8329,65.2820,-10.93,93.0,40.0,5.86,CA,2022-02-05 22:13:58-05:00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Get latitude and longitude from city locations
locations = weather_data[["latitude", "longitude"]]

# Get humidity values to use as weight
humidity = weather_data["humidity_percent"]

# Display locations and humidity 
print(locations)
print(humidity)

     latitude  longitude
0    -12.1167   -77.0500
1    -54.8000   -68.3000
2     65.6145   -37.6368
3     25.0865   -80.4473
4     65.2820  -126.8329
..        ...        ...
578  -32.5000   137.7667
579   -1.7656   -55.8661
580    4.8845    -1.7554
581   20.2167   -87.4667
582   13.0667   -59.5333

[583 rows x 2 columns]
0      50.0
1      57.0
2      55.0
3      80.0
4      93.0
       ... 
578    16.0
579    94.0
580    86.0
581    94.0
582    73.0
Name: humidity_percent, Length: 583, dtype: float64


In [5]:
# Plot heatmap of each city's humidity
fig = gmaps.figure()

# Heat layer
heat_layer = gmaps.heatmap_layer(
    locations, 
    weights=humidity, 
    dissipating=True, 
    max_intensity=200, 
    point_radius=15
)

# Add heat layer to figure
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# My ideal weather conditions: 70 <= map temp <= 85, 30 <= humidity <= 60, 
# 0 <= cloudiness <= 25

# Get new DataFrame based on my conditions
ideal_cities = weather_data.loc[
    (weather_data["maxtemp_f"] >= 70)
    & (weather_data["maxtemp_f"] <= 85) 
    & (weather_data["humidity_percent"] >= 30) 
    & (weather_data["humidity_percent"] <= 60)
    & (weather_data["cloudiness_percent"] >= 0)
    & (weather_data["cloudiness_percent"] <= 25)
]

# Drop rows with any NA values in its columns
ideal_cities = ideal_cities.dropna(axis=0, how="any")

# Reset index numbers for cleaner iteration later
ideal_cities = ideal_cities.reset_index()

# Display new DataFrame
ideal_cities

,index,city,longitude,latitude,maxtemp_f,humidity_percent,cloudiness_percent,windspeed_mph,country,date
0,37,Hobart,147.3294,-42.8794,76.01,46.0,20.0,1.01,AU,2022-02-05 22:20:17-05:00
1,83,New Norfolk,147.0587,-42.7826,76.66,35.0,0.0,1.99,AU,2022-02-05 22:21:32-05:00
2,145,Launceston,147.1667,-41.4500,77.61,40.0,0.0,4.00,AU,2022-02-05 22:23:46-05:00
3,217,Port Lincoln,135.8667,-34.7333,78.30,48.0,7.0,20.80,AU,2022-02-05 22:15:21-05:00
4,224,Bowen,148.2333,-20.0167,84.15,59.0,11.0,16.93,AU,2022-02-05 22:15:24-05:00
5,234,Zirándaro de los Chávez,-100.9833,18.4500,79.07,43.0,0.0,4.07,MX,2022-02-05 22:21:17-05:00
6,288,Bolitoc,119.8769,15.7395,82.98,55.0,14.0,3.87,PH,2022-02-05 22:27:22-05:00
7,375,Sur,59.5289,22.5667,70.45,44.0,0.0,7.29,OM,2022-02-05 22:21:49-05:00


### Hotel Map
* ~~Store into variable named `hotel_df`~~.
* **Store into `ideal_cities` DataFrame from previous cell.**
* ~~Add a "Hotel Name" column to the DataFrame.~~
* **Store into `lodging_name`, `lodging_long`, `lodging_lat` columns**
* Set parameters to search for ~~hotels~~ lodgings with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first ~~Hotel~~ lodging result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
# Base URL for Google Place Nearby Search request
nearby_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# Parameters for query - leave location as empty string to modify during loop
# NOTE: Turns out, not all of my ideal cities have hotels as accommodations...
# Use type=lodging and do not use keyword=hotel
params = {
    "location": "",
    "radius": "5000",
    "type": "lodging",
    "key": g_key
}

# Empty dictionary to keep track of each city's lodging name and coordinates to
# add to DataFrame later
lodging_info = {
    "lodging_names": [],
    "lodging_long": [],
    "lodging_lat": []
}

# For each row in the ideal_cities DataFrame, build query using coordinates
for index, row in ideal_cities.iterrows():

    # Get latitude and longitude from the row, build query string
    coord = f"{row['latitude']},{row['longitude']}"

    # Modify location key in params dictionary
    params["location"] = coord
    
    # Get response from API
    response = requests.get(nearby_url, params=params)

    # Get first hotel's name and coordinates from json object
    lodgings = response.json()
    lodging_name = lodgings["results"][0]["name"]
    lodging_long = lodgings["results"][0]["geometry"]["location"]["lng"]
    lodging_lat = lodgings["results"][0]["geometry"]["location"]["lat"]

    # Add hotel's name to lodging dictionary
    lodging_info["lodging_names"].append(lodging_name)

    # Add hotel's coordinates to lodging dictionary
    lodging_info["lodging_long"].append(lodging_long)
    lodging_info["lodging_lat"].append(lodging_lat)

# Add dictionary info to new columns in ideal_cities
ideal_cities["lodging_name"] = lodging_info["lodging_names"]
ideal_cities["lodging_long"] = lodging_info["lodging_long"]
ideal_cities["lodging_lat"] = lodging_info["lodging_lat"]

# Display updated ideal_cities with lodging info
ideal_cities

,index,city,longitude,latitude,maxtemp_f,humidity_percent,cloudiness_percent,windspeed_mph,country,date,lodging_name,lodging_long,lodging_lat
0,37,Hobart,147.3294,-42.8794,76.01,46.0,20.0,1.01,AU,2022-02-05 22:20:17-05:00,Mantra on Collins Hobart,147.330726,-42.880980
1,83,New Norfolk,147.0587,-42.7826,76.66,35.0,0.0,1.99,AU,2022-02-05 22:21:32-05:00,The Shingles Riverside Cottages,147.046962,-42.782249
2,145,Launceston,147.1667,-41.4500,77.61,40.0,0.0,4.00,AU,2022-02-05 22:23:46-05:00,Hotel Grand Chancellor Launceston,147.140569,-41.434489
3,217,Port Lincoln,135.8667,-34.7333,78.30,48.0,7.0,20.80,AU,2022-02-05 22:15:21-05:00,Port Lincoln Hotel,135.854491,-34.717831
4,224,Bowen,148.2333,-20.0167,84.15,59.0,11.0,16.93,AU,2022-02-05 22:15:24-05:00,Castle Motor Lodge Motel,148.234958,-20.008885
5,234,Zirándaro de los Chávez,-100.9833,18.4500,79.07,43.0,0.0,4.07,MX,2022-02-05 22:21:17-05:00,Hotel Pineda,-100.977249,18.475946
6,288,Bolitoc,119.8769,15.7395,82.98,55.0,14.0,3.87,PH,2022-02-05 22:27:22-05:00,Seasun Beach Resort & Hotel,119.885387,15.720082
7,375,Sur,59.5289,22.5667,70.45,44.0,0.0,7.29,OM,2022-02-05 22:21:49-05:00,Sur Plaza Hotel,59.495858,22.566435


In [35]:
# NOTE: code adjusted according my own column and variable names

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{lodging_name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
lodging_infobox = [info_box_template.format(**row) for index, row in ideal_cities.iterrows()]
marker_locations = ideal_cities[["lodging_lat", "lodging_long"]]

In [36]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(
    marker_locations, 
    info_box_content=lodging_infobox
)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))